# Projeto Estabilização

## Marcelo G. de Andrade

### Features do projeto

- [X] Estabilização usando fluxo do centro da imagem
- [X] Remover bordas pretas da estabilização mantendo aspect ratio
- [ ] Estabilização da rotação

In [2]:
def crop_center(img,cropx,cropy):
    y,x = img.shape
    startx = x//2 - cropx//2
    starty = y//2 - cropy//2    
    cropped = img[starty:starty+cropy, startx:startx+cropx]
    return cropped

def crop_black_bar(img):
    '''
    Baseado em : https://stackoverflow.com/questions/13538748/crop-black-edges-with-opencv
    '''
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
    _,contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    x,y,w,h = cv2.boundingRect(cnt)
    if (w / 640 > h / 480):
        w = int(h * (640/480))
    else:
        h = int(w * (480/640))
    crop = img[y:y+h,x:x+w]
    return cv2.resize(crop, (1280, 960))
    
def draw_rect(img, cropx, cropy):
    y,x, colors = img.shape
    startx = x//2 - cropx//2
    starty = y//2 - cropy//2    
    cv2.rectangle(img, (startx,starty), (startx+cropx, starty+cropy),(0,255,0),3 )
    return img

In [3]:
'''
 Based on the following tutorial:
   http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html
'''

import numpy as np
import cv2

SIZE = 200
prevFlow = np.zeros((SIZE,SIZE,2))
# Start the webcam
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture('../data/vtest.avi')

# Take the first frame and convert it to gray
ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Create the HSV color image
hsvImg = np.zeros_like(frame)
hsvImg[..., 1] = 255

# Play until the user decides to stop
while True:
    # Save the previous frame data
    previousGray = gray
     
    # Get the next frame
    ret , frame = cap.read()
    
    if ret:
        # Convert the frame to gray scale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Calculate the dense optical flow
        flow = cv2.calcOpticalFlowFarneback(crop_center(previousGray, SIZE, SIZE), crop_center(gray, SIZE, SIZE) , None, 0.5, 3, 15, 3, 5, 1.2, 0)
        prevFlow += flow
        
        mean = np.mean(prevFlow, axis=1)
        mean = np.mean(mean, axis=0)
        
        translation_matrix = np.float32([ [1,0,-mean[0]], [0,1,-mean[1]] ])
        num_rows, num_cols = frame.shape[:2]
        img_translation = cv2.warpAffine(frame, translation_matrix, (num_cols, num_rows))
        
        # Display the resulting frame
        img2 = cv2.resize(img_translation, (1280, 960))   
        cv2.imshow('dense optical flow',crop_black_bar(img_translation))
        k = cv2.waitKey(30) & 0xff
        
        # Exit if the user press ESC
        if k == 27:
            break
    else:
        break

# When everything is done, release the capture and close all windows
cap.release()
cv2.destroyAllWindows()